### P5 - DETECCION Y CARACTERIZACION DE CARAS

In [ ]:
import cv2
import mediapipe as mp

mp_face_mesh = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils

cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)

def process_frame(frame, face_mesh):
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(frame_rgb)

    if results.multi_face_landmarks is not None:
        for face_landmarks in results.multi_face_landmarks:
            mp_drawing.draw_landmarks(frame, face_landmarks,
                                      mp_face_mesh.FACEMESH_TESSELATION,
                                      mp_drawing.DrawingSpec(color=(0, 255, 255), thickness=1, circle_radius=1),
                                      mp_drawing.DrawingSpec(color=(255, 0, 255), thickness=1))
            points = [127, 356, 183, 332, 0, 17, 468, 473]
            for point in points:
                if point < len(face_landmarks.landmark):  # Check if the index is valid
                    x = int(face_landmarks.landmark[point].x * frame.shape[1])
                    y = int(face_landmarks.landmark[point].y * frame.shape[0])
                    cv2.circle(frame, (x, y), 2, (0, 255, 0), -1)
    return frame

with mp_face_mesh.FaceMesh(
    static_image_mode=False,
    max_num_faces=1,
    min_detection_confidence=0.5) as face_mesh:

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.flip(frame, 1)
        frame = process_frame(frame, face_mesh)

        cv2.imshow("Frame", frame)
        k = cv2.waitKey(1) & 0xFF
        if k == 27:
            break
cap.release()
cv2.destroyAllWindows()



In [ ]:
umbral = 20


if(p[17] - p[1] > umbral):
    print("Boca abierta")